In [1]:
# script to compute ignition delays for 50 mechanisms of top parameters

In [2]:
import os
import cantera as ct
import numpy as np
import pandas as pd
import concurrent.futures


import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Load the experimental conditions
ignition_delay_data = '/work/westgroup/harris.se/autoscience/autoscience/butane/experimental_data/butane_ignition_delay.csv'
ignition_delay_data = '/home/moon/autoscience/autoscience/butane/experimental_data/butane_ignition_delay.csv'
df_exp = pd.read_csv(ignition_delay_data)

# slice just table 7, where phi=1.0
table7 = df_exp[df_exp['Table'] == 7]
# Define Initial conditions using experimental data
tau7 = table7['time (ms)'].values.astype(float)  # ignition delay
T7 = table7['T_C'].values  # Temperatures
P7 = table7['nominal pressure(atm)'].values * ct.one_atm  # pressures in atm


# list of starting conditions
# Mixture compositions taken from table 2 of
# https://doi-org.ezproxy.neu.edu/10.1016/j.combustflame.2010.01.016
concentrations = []
# for phi = 1
x_diluent = 0.7649
conc_dict = {
    'O2(2)': 0.2038,
    'butane(1)': 0.03135
}

for i in range(0, len(table7)):
    x_N2 = table7['%N2'].values[i] / 100.0 * x_diluent
    x_Ar = table7['%Ar'].values[i] / 100.0 * x_diluent
    x_CO2 = table7['%CO2'].values[i] / 100.0 * x_diluent
    conc_dict['N2'] = x_N2
    conc_dict['Ar'] = x_Ar
    conc_dict['CO2(7)'] = x_CO2
    concentrations.append(conc_dict)


In [4]:
# Take Reactor Conditions from Table 7 of supplementary info in
# https://doi-org.ezproxy.neu.edu/10.1016/j.combustflame.2010.01.016
def get_delay(gas, T, P, X):
    # function to run a RCM simulation

    t_end = 1.0  # time in seconds
    gas.TPX = T, P, X

    env = ct.Reservoir(ct.Solution('air.yaml'))
    # env = ct.Reservoir(ct.Solution('air.xml'))
    reactor = ct.IdealGasReactor(gas)
    wall = ct.Wall(reactor, env, A=1.0, velocity=0)
    reactor_net = ct.ReactorNet([reactor])
    # # allegedly faster solving
    # reactor_net.derivative_settings = {"skip-third-bodies": True, "skip-falloff": True}
    # reactor_net.preconditioner = ct.AdaptivePreconditioner()

    times = [0]
    T = [reactor.T]
    P = [reactor.thermo.P]
    X = [reactor.thermo.X]  # mol fractions
    while reactor_net.time < t_end:
        reactor_net.step()

        times.append(reactor_net.time)
        T.append(reactor.T)
        P.append(reactor.thermo.P)
        X.append(reactor.thermo.X)

    slopes = np.gradient(P, times)
    i = np.argmax(slopes)
    return times[i]

In [5]:
def run_simulation(condition_index):
    gas = ct.Solution(cti_path)
    X = concentrations[condition_index]
    delay = get_delay(gas, T7[condition_index], P7[condition_index], X)
    print(f'Completed {condition_index}:\t {delay}')
    return delay

In [6]:
save_dir = '/home/moon/autoscience/reaction_calculator/error_vs_num_included/results'
mech_dir = '/home/moon/autoscience/reaction_calculator/error_vs_num_included/mechs'

In [8]:
for i in range(0, 51):
    print(f'running mechanism {i}')
    # load the CTI
    cti_path = os.path.join(mech_dir, f'top_{i}.cti')
    if not os.path.exists(cti_path):
        continue
    gas = ct.Solution(cti_path)
    
    # Run all simulations in parallel
    delays = np.zeros(len(table7))
    condition_indices = np.arange(0, len(table7))
    with concurrent.futures.ProcessPoolExecutor(max_workers=16) as executor:
        for condition_index, delay_time in zip(condition_indices, executor.map(run_simulation, condition_indices)):
            delays[condition_index] = delay_time
    
    np.save(os.path.join(save_dir, f'delays_{i}.npy'), delays)

running mechanism 0
Completed 13:	 0.024629607815479616
Completed 15:	 0.013983329164772643
Completed 14:	 0.023349025151091538
Completed 6:	 0.14791667636256164
Completed 1:	 0.04854438699501166Completed 3:	 0.04868176194585489
Completed 10:	 0.06520822884244994

Completed 9:	 0.09524339850848947Completed 12:	 0.03535775087192712

Completed 4:	 0.07558799561746257
Completed 8:	 0.10859170352272628
Completed 11:	 0.05563236520895804
Completed 7:	 0.1485753452189033
Completed 5:	 0.07737167394281456
Completed 0:	 0.049083278068407024
Completed 2:	 0.048144458947842486
running mechanism 1
running mechanism 2
Completed 6:	 0.1246939937491167
Completed 15:	 0.013996607920455895
Completed 12:	 0.03530453377130646
Completed 14:	 0.02335823585615054
Completed 13:	 0.024635717316313213Completed 2:	 0.048706223290354345
Completed 11:	 0.05524446979764865Completed 8:	 0.10469297343500113


Completed 0:	 0.05524333214272763
Completed 10:	 0.06452572816844132
Completed 5:	 0.06807845283850432
Comp

Completed 7:	 0.11299616452355951
Completed 14:	 0.02303627913093467
Completed 6:	 0.11183069020416274Completed 13:	 0.02429054114422307

Completed 12:	 0.03472546598539706
Completed 5:	 0.0624414708079485
Completed 15:	 0.013815007127646176
Completed 9:	 0.08946088801041266
Completed 10:	 0.06291304112873962
Completed 3:	 0.047277005293479786
Completed 1:	 0.054576456020789234
Completed 2:	 0.04706275105004257
Completed 0:	 0.05533523710875753
Completed 4:	 0.0614666743642295
running mechanism 18
Completed 13:	 0.02429061798326116Completed 15:	 0.013815019297525365

Completed 14:	 0.023036344191482584Completed 12:	 0.03472570140570377

Completed 10:	 0.06291452608039837
Completed 2:	 0.047067189017747965
Completed 3:	 0.04728146624888907Completed 8:	 0.10033431405650615

Completed 11:	 0.054027822106906716
Completed 9:	 0.0894654112826361
Completed 5:	 0.06244777454687523
Completed 6:	 0.11184609442140168
Completed 7:	 0.11301176208863137
Completed 1:	 0.054581441304774485
Completed 4

Completed 13:	 0.02429025728290329
Completed 9:	 0.08946743389753385
Completed 8:	 0.10033779708627784
Completed 6:	 0.11185763657544463
Completed 0:	 0.05534079809027589
Completed 1:	 0.05458261176054988
Completed 12:	 0.03472535912778159
Completed 5:	 0.062456288022215055
Completed 15:	 0.013814481480253347
Completed 10:	 0.06291467921602222
Completed 11:	 0.0540277005034254
Completed 3:	 0.04728867798585283
Completed 2:	 0.04707439373779256
Completed 4:	 0.06148140548060284
running mechanism 39
running mechanism 40
Completed 11:	 0.054027700409860276
Completed 6:	 0.11185763644219092
Completed 8:	 0.10033779730964114
Completed 5:	 0.06245628766619556
Completed 10:	 0.0629146791143814
Completed 9:	 0.08946743366804558
Completed 2:	 0.04707439353098041
Completed 12:	 0.034725359168026916
Completed 7:	 0.11302345289193207
Completed 15:	 0.013814481569905238
Completed 1:	 0.05458261206993126
Completed 14:	 0.02303597795284204
Completed 13:	 0.02429025722006246
Completed 4:	 0.0614814055